In [4]:
import pandas as pd

## Esta Notebook construye muestreos del Censo 2010

Para correrla necesita tener los archivos de la base de Datos del Censo 2010.  

   - La base de Datos en formato REDATAM disponible en el repositorio no oficial de datos abierto (DatAR)

http://datar.info/dataset/censo-nacional-de-poblacion-hogares-y-viviendas-2010-cd-redatam

   - Un convertidor de REDATAM (funciona en Windows) disponible en:
    
https://www.aacademica.org/conversor.redatam

In [5]:
## El path donde tenemos los archivos PERSONA, HOGAR y VIVIENDA
censo_DB_path = '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub'

### Poblacion por departamento (Comuna, Partido) en 2001 - 2025

In [6]:

# El archivo 'proy_pop200125.csv' contiene la informacion oficial de proyecciones de poblacion por departamento publicada por INDEC
#  ('https://www.indec.gob.ar/ftp/cuadros/poblacion/proyeccion_departamentos_10_25.pdf')
proy_pop = pd.read_csv('./../data/info/proy_pop200125.csv', encoding = 'utf-8')
proy_pop.head(2)
# proy_pop.sample(5).iloc[:, 2:].T.plot()

,DPTO,NOMDPTO,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,2001,Comuna 01,223620,225878,228137,230395,232654,234912,237171,239429,...,252053,253271,254408,255457,256405,257235,257934,258497,258922,259205
1,2002,Comuna 02,138156,139551,140947,142342,143737,145133,146528,147923,...,149848,149720,149607,149510,149430,149371,149337,149328,149345,149389


### Referencia de radios censales 2010

In [7]:
## Referencia de radios censales segun Censo 2010
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
radio_ref['DPTO'] = radio_ref['DPTO'].astype(int)
radio_ref['PROV'] = radio_ref['PROV'].astype(int)

In [12]:
radio_ref[['DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/info/dptos.csv', index = False)
radio_ref[['PROV', 'NOMPROV']].drop_duplicates().to_csv('./../data/info/provs.csv', index = False)

### Personas por hogar (columna IX_TOT)

In [8]:
import os

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [6]:
# if not os.path.exists('./../data/info/IX_TOT.csv'):
#     # Cantidad de personas  en cada hogar. No esta la columna pero la podemos construir..
#     PERSONA = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/PERSONA.csv', sep = ';', usecols = ['HOGAR_REF_ID']) 
#     # Los datasets csv del censo fueron extraidos en ext_CPV2010_basico_radio_pub
#     # Pesa algunos Gb, por eso se carga con dask.

#     with ProgressBar():
#         IX_TOT = PERSONA['HOGAR_REF_ID'].value_counts().reset_index().compute()
#         IX_TOT.columns = ['HOGAR_REF_ID', 'IX_TOT']

#     IX_TOT.to_csv('./../data/info/IX_TOT.csv', index = False)

# else:
#     IX_TOT = pd.read_csv('./../data/info/IX_TOT.csv')

# IX_TOT.head() # el tamanio de 12.197.647 hogares

,HOGAR_REF_ID,IX_TOT
0,746811,1694
1,5317447,1628
2,10234703,1520
3,1602541,1410
4,8717569,1373


In [7]:
# radio_ref = pd.read_csv('./../Censo_individual/data/radio_ref.csv')

## Hogar y Vivienda

In [9]:
# frac = 0.01

# #Esto es para extraer las viviendas, hogares y personas de los partidos (DPTOs) en cuestion.
# # seleccion_DPTOS y usecols nos sirven para no cargar data innecesaria.
# VIVIENDA = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/VIVIENDA.csv', sep = ';', usecols = ['VIVIENDA_REF_ID', 'RADIO_REF_ID', 'TIPVV', 'V01'])
# VIVIENDA = VIVIENDA.merge(radio_ref[['RADIO_REF_ID', 'DPTO']])
# # VIVIENDA_ = VIVIENDA.loc[VIVIENDA.DPTO.isin(seleccion_DPTOS)]
    
# HOGAR = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/HOGAR.csv', sep = ';', usecols = ['HOGAR_REF_ID', 'VIVIENDA_REF_ID']) # csv is too big, so it is dask-loaded. Not sure it's efficient thou

# with ProgressBar():
#     HOGAR_DPTO = HOGAR.merge(VIVIENDA[['VIVIENDA_REF_ID', 'DPTO']]).compute()

[########################################] | 100% Completed |  1min 38.6s


In [40]:
seleccion_distritos = ['02', '04']

if len(np.unique([len(dist) for dist in seleccion_distritos])) != 1:
    print('Imputar solo provincias o bien solo departamentos.')
    
nivel = np.unique([len(dist) for dist in seleccion_distritos])[0]



2

In [ ]:
frac = 0.01

#Esto es para extraer las viviendas, hogares y personas de los partidos (DPTOs) en cuestion.
# seleccion_DPTOS y usecols nos sirven para no cargar data innecesaria.
VIVIENDA = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/VIVIENDA.csv', sep = ';', usecols = ['VIVIENDA_REF_ID', 'RADIO_REF_ID', 'TIPVV', 'V01'])
VIVIENDA = VIVIENDA.merge(radio_ref[['RADIO_REF_ID', 'DPTO', 'PROV']])

if nivel == 2:
    VIVIENDA = VIVIENDA.loc[VIVIENDA.PROV.isin(seleccion_distritos)]
    
elif nivel == 4:
    VIVIENDA = VIVIENDA.loc[VIVIENDA.DPTO.isin(seleccion_distritos)]
    
HOGAR = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/HOGAR.csv', sep = ';', usecols = ['HOGAR_REF_ID', 'VIVIENDA_REF_ID']) # csv is too big, so it is dask-loaded. Not sure it's efficient thou

with ProgressBar():
    HOGAR_DPTO = HOGAR.merge(VIVIENDA[['VIVIENDA_REF_ID', 'DPTO']]).compute()

In [9]:
# # poblacion 2010 por depto
# # Personas
# P02 = pd.read_csv('./../data/info/PERSONA-P02.csv', encoding = 'latin-1')

# P02['DPTO'] = P02['radio'].astype(str).str.zfill(9).str[:5].astype(int)
# PERSONAS = P02.groupby('DPTO')[['TOTAL']].sum()

# # Hogares
# HOGARES = HOGAR_DPTO.groupby(['DPTO']).nunique()

# poblacion = HOGARES.join(PERSONAS)

In [44]:
name = 'ARG'

In [49]:
frac

0.01

In [47]:
# grouped = HOGAR_DPTO.merge(proyeccion[['DPTO', 'ratio_18']]).groupby('DPTO')
# ratios = proy_pob.div(proy_pob['2010'], 0).reset_index()
ratios = proy_pop.set_index(['DPTO', 'NOMDPTO']).div(proy_pop['2010'].values, 0).reset_index()

PERSONA = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/PERSONA.csv', sep = ';', usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID', 'P01', 'P02', 'P03', 'P05', 'P06',
       'P07', 'P12', 'P08', 'P09', 'P10', 'CONDACT'])
    
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    grouped = HOGAR_DPTO.merge(ratios[['DPTO', yr]]).groupby('DPTO')
    sample = grouped.apply(lambda x: x.sample(frac=frac*x[yr].mean()))

    HOGAR = dd.read_csv(censo_DB_path + 'ext_CPV2010_basico_radio_pub/HOGAR.csv', sep = ';', usecols = ['HOGAR_REF_ID', 'VIVIENDA_REF_ID', 'H05', 'H06', 'H07', 'H08',
           'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP', 'TOTPERS']) 

    VIVIENDA_sample = VIVIENDA.loc[VIVIENDA.VIVIENDA_REF_ID.isin(sample.VIVIENDA_REF_ID)]
    HOGAR_sample = HOGAR.loc[HOGAR.HOGAR_REF_ID.isin(sample.HOGAR_REF_ID)]

    tabla_censo = VIVIENDA_sample.merge(HOGAR_sample)
#     tabla_censo = tabla_censo.merge(IX_TOT) # se mergea mas adelante

    with ProgressBar():
        table = tabla_censo.compute()

    # Adaptamos las categorias de respuestas para que queden iguales a las de la EPH
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})

    # saber de que aglo es el hogar.
    table = table.merge(radio_ref[['RADIO_REF_ID','AGLOMERADO']]) 

# #    Option to save table now, in case it crashes at this step.
#     if not os.path.exists('./../data/yr_samples/'):
#         os.makedirs('./../data/yr_samples/')
#     table.to_csv('./../data/yr_samples/sample_censo_table_f'+str(frac)+'_'+yr+'_ARG.csv', index = False)

    PERSONA_sample = PERSONA.loc[PERSONA.HOGAR_REF_ID.isin(sample.HOGAR_REF_ID)]


    with ProgressBar():
        table = table.merge(PERSONA_sample.compute())

    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})
    
    ## Calcula tamano del hogar
    IX_TOT = table['HOGAR_REF_ID'].value_counts().reset_index(); IX_TOT.columns = ['HOGAR_REF_ID', 'IX_TOT_']
    table = table.merge(IX_TOT)

    # Guardar sampleo del censo
    
    if not os.path.exists('./../data/censo_samples/'):
        os.makedirs('./../data/censo_samples/')
        
    table.to_csv('./../data/censo_samples/table_f'+str(frac)+'_'+yr+'_'+name+'.csv', index = False)

2020


KeyboardInterrupt: 

In [24]:
# table_ = pd.read_csv('./../../yr_samples/sample_censo_table_f0.02_2020_ARG.csv')